<a href="https://colab.research.google.com/github/jimmymanianchira/RandomExperiments/blob/master/Horovod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

## Failed Experimentation of working with Horovod. Need to figure out why this failed


In [0]:
!pip install h5py >=2.9.0

In [0]:
import h5py
h5py.__version__

'2.9.0'

In [0]:
!pip install horovod
!pip install pyspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark import SparkConf, Row
import datetime
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F


In [0]:
def fn(magic_number):
  import horovod.torch as hvd
  hvd.init()
  print('Hello, rank = %d, local_rank = %d, size = %d, local_size = %d, magic_number = %d' % (hvd.rank(), hvd.local_rank(), hvd.size(), hvd.local_size(), magic_number))
  return hvd.rank()
...

Ellipsis

In [0]:
import horovod.spark
horovod.spark.run(fn, args=(42,))

Running 2 processes (inferred from spark.default.parallelism)...
[1,0]<stdout>:Hello, rank = 0, local_rank = 0, size = 2, local_size = 2, magic_number = 42
[1,1]<stdout>:Hello, rank = 1, local_rank = 1, size = 2, local_size = 2, magic_number = 42


[1b280e9a30cd:02080] PMIX ERROR: NO-PERMISSIONS in file src/dstore/pmix_esh.c at line 530


[0, 1]

googletrend.csv
sample_submission.csv
state_names.csv
store.csv
store_states.csv
test.csv
train.csv
weather.csv


In [0]:
!wget http://files.fast.ai/part2/lesson14/rossmann.tgz 

--2019-06-04 06:31:15--  http://files.fast.ai/part2/lesson14/rossmann.tgz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7730448 (7.4M) [text/plain]
Saving to: ‘rossmann.tgz.1’

rossmann.tgz.1      100%[===================>]   7.37M  48.4MB/s    in 0.2s    

2019-06-04 06:31:16 (48.4 MB/s) - ‘rossmann.tgz.1’ saved [7730448/7730448]



In [0]:
!tar zxvf rossmann.tgz

googletrend.csv
sample_submission.csv
state_names.csv
store.csv
store_states.csv
test.csv
train.csv
weather.csv


In [0]:
import io
import os
import pyarrow as pa
from pyspark import SparkConf, Row
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [0]:
LOCAL_SUBMISSION_CSV = 'submission.csv'
LOCAL_CHECKPOINT_FILE = 'checkpoint.h5'

In [0]:
LIGHT_PROCESSING_CLUSTER = None  # or 'spark://hostname:7077'
TRAINING_CLUSTER = None  # or 'spark://hostname:7077'

# The number of training processes.
NUM_TRAINING_PROC = 4

# Desired sampling rate.  Useful to set to low number (e.g. 0.01) to make sure
# that end-to-end process works.
SAMPLE_RATE = None  # or use 0.01

# Batch size & learning rate to use.
BATCH_SIZE = 100
LR = 1e-4

# HDFS driver to use with Petastorm.
PETASTORM_HDFS_DRIVER = 'libhdfs'

In [0]:
conf = SparkConf().setAppName('data_prep').set('spark.sql.shuffle.partitions', '16')
if LIGHT_PROCESSING_CLUSTER:
    conf.setMaster(LIGHT_PROCESSING_CLUSTER)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [0]:
!ls

'=2.9.0'		 spark-2.4.3-bin-hadoop2.7	   test.csv
 googletrend.csv	 spark-2.4.3-bin-hadoop2.7.tgz	   test_df.parquet
 rossmann.tgz		 spark-2.4.3-bin-hadoop2.7.tgz.1   train.csv
 rossmann.tgz.1		 state_names.csv		   train_df.parquet
 sample_data		 store.csv			   val_df.parquet
 sample_submission.csv	 store_states.csv		   weather.csv


In [0]:
train_csv = spark.read.csv('train.csv', header=True)
test_csv = spark.read.csv('test.csv', header=True)

In [0]:
store_csv = spark.read.csv('store.csv', header=True)
store_states_csv = spark.read.csv('store_states.csv', header=True)
state_names_csv = spark.read.csv('state_names.csv' , header=True)
google_trend_csv = spark.read.csv('googletrend.csv' , header=True)
weather_csv = spark.read.csv('weather.csv' , header=True)

In [0]:
def expand_date(df):
    df = df.withColumn('Date', df.Date.cast(T.DateType()))
    return df \
        .withColumn('Year', F.year(df.Date)) \
        .withColumn('Month', F.month(df.Date)) \
        .withColumn('Week', F.weekofyear(df.Date)) \
        .withColumn('Day', F.dayofmonth(df.Date))


def prepare_google_trend():
    # Extract week start date and state.
    google_trend_all = google_trend_csv \
        .withColumn('Date', F.regexp_extract(google_trend_csv.week, '(.*?) -', 1)) \
        .withColumn('State', F.regexp_extract(google_trend_csv.file, 'Rossmann_DE_(.*)', 1))

    # Map state NI -> HB,NI to align with other data sources.
    google_trend_all = google_trend_all \
        .withColumn('State', F.when(google_trend_all.State == 'NI', 'HB,NI').otherwise(google_trend_all.State))

    # Expand dates.
    return expand_date(google_trend_all)


def add_elapsed(df, cols):
    def add_elapsed_column(col, asc):
        def fn(rows):
            last_store, last_date = None, None
            for r in rows:
                if last_store != r.Store:
                    last_store = r.Store
                    last_date = r.Date
                if r[col]:
                    last_date = r.Date
                fields = r.asDict().copy()
                fields[('After' if asc else 'Before') + col] = (r.Date - last_date).days
                yield Row(**fields)
        return fn

    df = df.repartition(df.Store)
    for asc in [False, True]:
        sort_col = df.Date.asc() if asc else df.Date.desc()
        rdd = df.sortWithinPartitions(df.Store.asc(), sort_col).rdd
        for col in cols:
            rdd = rdd.mapPartitions(add_elapsed_column(col, asc))
        df = rdd.toDF()
    return df


def prepare_df(df):
    num_rows = df.count()

    # Expand dates.
    df = expand_date(df)

    df = df \
        .withColumn('Open', df.Open != '0') \
        .withColumn('Promo', df.Promo != '0') \
        .withColumn('StateHoliday', df.StateHoliday != '0') \
        .withColumn('SchoolHoliday', df.SchoolHoliday != '0')

    # Merge in store information.
    store = store_csv.join(store_states_csv, 'Store')
    df = df.join(store, 'Store')

    # Merge in Google Trend information.
    google_trend_all = prepare_google_trend()
    df = df.join(google_trend_all, ['State', 'Year', 'Week']).select(df['*'], google_trend_all.trend)

    # Merge in Google Trend for whole Germany.
    google_trend_de = google_trend_all[google_trend_all.file == 'Rossmann_DE']
    df = df.join(google_trend_de, ['Year', 'Week']).select(df['*'], google_trend_all.trend.alias('trend_de'))

    # Merge in weather.
    weather = weather_csv.join(state_names_csv, weather_csv.file == state_names_csv.StateName)
    df = df.join(weather, ['State', 'Date'])

    # Fix null values.
    df = df \
        .withColumn('CompetitionOpenSinceYear', F.coalesce(df.CompetitionOpenSinceYear, F.lit(1900))) \
        .withColumn('CompetitionOpenSinceMonth', F.coalesce(df.CompetitionOpenSinceMonth, F.lit(1))) \
        .withColumn('Promo2SinceYear', F.coalesce(df.Promo2SinceYear, F.lit(1900))) \
        .withColumn('Promo2SinceWeek', F.coalesce(df.Promo2SinceWeek, F.lit(1)))

    # Days & months competition was open, cap to 2 years.
    df = df.withColumn('CompetitionOpenSince',
                       F.to_date(F.format_string('%s-%s-15', df.CompetitionOpenSinceYear,
                                                 df.CompetitionOpenSinceMonth)))
    df = df.withColumn('CompetitionDaysOpen',
                       F.when(df.CompetitionOpenSinceYear > 1900,
                              F.greatest(F.lit(0), F.least(F.lit(360 * 2), F.datediff(df.Date, df.CompetitionOpenSince))))
                       .otherwise(0))
    df = df.withColumn('CompetitionMonthsOpen', (df.CompetitionDaysOpen / 30).cast(T.IntegerType()))

    # Days & weeks of promotion, cap to 25 weeks.
    df = df.withColumn('Promo2Since',
                       F.expr('date_add(format_string("%s-01-01", Promo2SinceYear), (Promo2SinceWeek - 1) * 7)'))
    df = df.withColumn('Promo2Days',
                       F.when(df.Promo2SinceYear > 1900,
                              F.greatest(F.lit(0), F.least(F.lit(25 * 7), F.datediff(df.Date, df.Promo2Since))))
                       .otherwise(0))
    df = df.withColumn('Promo2Weeks', (df.Promo2Days / 7).cast(T.IntegerType()))

    # Check that we did not lose any rows through inner joins.
    assert num_rows == df.count(), 'lost rows in joins'
    return df


def build_vocabulary(df, cols):
    vocab = {}
    for col in cols:
        values = [r[0] for r in df.select(col).distinct().collect()]
        col_type = type([x for x in values if x is not None][0])
        default_value = col_type()
        vocab[col] = sorted(values, key=lambda x: x or default_value)
    return vocab


def cast_columns(df, cols):
    for col in cols:
        df = df.withColumn(col, F.coalesce(df[col].cast(T.FloatType()), F.lit(0.0)))
    return df


def lookup_columns(df, vocab):
    def lookup(mapping):
        def fn(v):
            return mapping.index(v)
        return F.udf(fn, returnType=T.IntegerType())

    for col, mapping in vocab.items():
        df = df.withColumn(col, lookup(mapping)(df[col]))
    return df

In [0]:
if SAMPLE_RATE:
    train_csv = train_csv.sample(withReplacement=False, fraction=SAMPLE_RATE)
    test_csv = test_csv.sample(withReplacement=False, fraction=SAMPLE_RATE)

In [0]:
train_df = prepare_df(train_csv).cache()
test_df = prepare_df(test_csv).cache()

# Add elapsed times from holidays & promos, the data spanning training & test datasets.
elapsed_cols = ['Promo', 'StateHoliday', 'SchoolHoliday']
elapsed = add_elapsed(train_df.select('Date', 'Store', *elapsed_cols)
                              .unionAll(test_df.select('Date', 'Store', *elapsed_cols)),
                      elapsed_cols)

# Join with elapsed times.
train_df = train_df \
    .join(elapsed, ['Date', 'Store']) \
    .select(train_df['*'], *[prefix + col for prefix in ['Before', 'After'] for col in elapsed_cols])
test_df = test_df \
    .join(elapsed, ['Date', 'Store']) \
    .select(test_df['*'], *[prefix + col for prefix in ['Before', 'After'] for col in elapsed_cols])

In [0]:
train_df = train_df.filter(train_df.Sales > 0)

print('===================')
print('Prepared data frame')
print('===================')
train_df.show()

categorical_cols = [
    'Store', 'State', 'DayOfWeek', 'Year', 'Month', 'Day', 'Week', 'CompetitionMonthsOpen', 'Promo2Weeks', 'StoreType',
    'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'Events', 'Promo',
    'StateHoliday', 'SchoolHoliday'
]

continuous_cols = [
    'CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Max_Humidity',
    'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
    'BeforePromo', 'AfterPromo', 'AfterStateHoliday', 'BeforeStateHoliday', 'BeforeSchoolHoliday', 'AfterSchoolHoliday'
]

all_cols = categorical_cols + continuous_cols

Prepared data frame
+-----+----------+-----+---------+-----+---------+----+-----+------------+-------------+----+-----+----+---+---------+----------+-------------------+-------------------------+------------------------+------+---------------+---------------+----------------+-----+--------+------------------+----------------+-----------------+----------------+----------+--------------+-------------+------------+-------------+------------+-------------------------+--------------------------+-------------------------+----------------+-----------------+----------------+------------------+-------------------+------------------+---------------+----------+--------+--------------+------------------+--------------------+-------------------+---------------------+-----------+----------+-----------+-----------+------------------+-------------------+----------+-----------------+------------------+
|State|      Date|Store|DayOfWeek|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|Year|Month|We

In [0]:
train_df = train_df.select(*(all_cols + ['Sales', 'Date'])).cache()
test_df = test_df.select(*(all_cols + ['Id', 'Date'])).cache()

# Build vocabulary of categorical columns.
vocab = build_vocabulary(train_df.select(*categorical_cols)
                                 .unionAll(test_df.select(*categorical_cols)).cache(),
                         categorical_cols)

# Cast continuous columns to float & lookup categorical columns.
train_df = cast_columns(train_df, continuous_cols + ['Sales'])
train_df = lookup_columns(train_df, vocab)
test_df = cast_columns(test_df, continuous_cols)
test_df = lookup_columns(test_df, vocab)

# Split into training & validation.
# Test set is in 2015, use the same period in 2014 from the training set as a validation set.
test_min_date = test_df.agg(F.min(test_df.Date)).collect()[0][0]
test_max_date = test_df.agg(F.max(test_df.Date)).collect()[0][0]
a_year = datetime.timedelta(365)
val_df = train_df.filter((test_min_date - a_year <= train_df.Date) & (train_df.Date < test_max_date - a_year))
train_df = train_df.filter((train_df.Date < test_min_date - a_year) | (train_df.Date >= test_max_date - a_year))

# Determine max Sales number.
max_sales = train_df.agg(F.max(train_df.Sales)).collect()[0][0]

print('===================================')
print('Data frame with transformed columns')
print('===================================')
train_df.show()

print('================')
print('Data frame sizes')
print('================')
train_rows, val_rows, test_rows = train_df.count(), val_df.count(), test_df.count()
print('Training: %d' % train_rows)
print('Validation: %d' % val_rows)
print('Test: %d' % test_rows)



Data frame with transformed columns
+-----+-----+---------+----+-----+---+----+---------------------+-----------+---------+----------+-------------+------------------------+---------------+------+-----+------------+-------------+-------------------+----------------+-----------------+----------------+------------+-------------+------------+------------------+-------------------+----------+-----+--------+-----------+----------+-----------------+------------------+-------------------+------------------+-------+----------+
|Store|State|DayOfWeek|Year|Month|Day|Week|CompetitionMonthsOpen|Promo2Weeks|StoreType|Assortment|PromoInterval|CompetitionOpenSinceYear|Promo2SinceYear|Events|Promo|StateHoliday|SchoolHoliday|CompetitionDistance|Max_TemperatureC|Mean_TemperatureC|Min_TemperatureC|Max_Humidity|Mean_Humidity|Min_Humidity|Max_Wind_SpeedKm_h|Mean_Wind_SpeedKm_h|CloudCover|trend|trend_DE|BeforePromo|AfterPromo|AfterStateHoliday|BeforeStateHoliday|BeforeSchoolHoliday|AfterSchoolHoliday|  Sale

In [0]:
# Save data frames as Parquet files.
train_df.write.parquet('train_df.parquet' , mode='overwrite')
val_df.write.parquet('val_df.parquet', mode='overwrite')
test_df.write.parquet('test_df.parquet' , mode='overwrite')

spark.stop()


print('==============')
print('Model training')
print('==============')

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Concatenate, Dense, Flatten, Reshape, BatchNormalization, Dropout
import tensorflow.keras.backend as K
import horovod.spark
import horovod.tensorflow.keras as hvd

Model training


In [0]:
def exp_rmspe(y_true, y_pred):
    """Competition evaluation metric, expects logarithic inputs."""
    pct = tf.square((tf.exp(y_true) - tf.exp(y_pred)) / tf.exp(y_true))
    # Compute mean excluding stores with zero denominator.
    x = tf.reduce_sum(tf.where(y_true > 0.001, pct, tf.zeros_like(pct)))
    y = tf.reduce_sum(tf.where(y_true > 0.001, tf.ones_like(pct), tf.zeros_like(pct)))
    return tf.sqrt(x / y)


def act_sigmoid_scaled(x):
    """Sigmoid scaled to logarithm of maximum sales scaled by 20%."""
    return tf.nn.sigmoid(x) * tf.log(max_sales) * 1.2


CUSTOM_OBJECTS = {'exp_rmspe': exp_rmspe,
                  'act_sigmoid_scaled': act_sigmoid_scaled}


def serialize_model(model):
    """Serialize model into byte array."""
    bio = io.BytesIO()
    with h5py.File(bio) as f:
        model.save(f)
    return bio.getvalue()


def deserialize_model(model_bytes, load_model_fn):
    """Deserialize model from byte array."""
    bio = io.BytesIO(model_bytes)
    with h5py.File(bio) as f:
        return load_model_fn(f, custom_objects=CUSTOM_OBJECTS)


# Do not use GPU for the session creation.
config = tf.ConfigProto(device_count={'GPU': 0})
K.set_session(tf.Session(config=config))

# Build the model.
inputs = {col: Input(shape=(1,), name=col) for col in all_cols}
embeddings = [Embedding(len(vocab[col]), 10, input_length=1, name='emb_' + col)(inputs[col])
              for col in categorical_cols]
continuous_bn = Concatenate()([Reshape((1, 1), name='reshape_' + col)(inputs[col])
                               for col in continuous_cols])
continuous_bn = BatchNormalization()(continuous_bn)
x = Concatenate()(embeddings + [continuous_bn])
x = Flatten()(x)
x = Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00005))(x)
x = Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00005))(x)
x = Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00005))(x)
x = Dense(500, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.00005))(x)
x = Dropout(0.5)(x)
output = Dense(1, activation=act_sigmoid_scaled)(x)
model = tf.keras.Model([inputs[f] for f in all_cols], output)
model.summary()

# Horovod: add Distributed Optimizer.
opt = tf.keras.optimizers.Adam(lr=LR, epsilon=1e-3)
opt = hvd.DistributedOptimizer(opt)
model.compile(opt, 'mae', metrics=[exp_rmspe])
model_bytes = serialize_model(model)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
CompetitionDistance (InputLayer (None, 1)            0                                            
__________________________________________________________________________________________________
Max_TemperatureC (InputLayer)   (None, 1)            0                                            
__________________________________________________________________________________________________
Mean_TemperatureC (InputLayer)  (None, 1)            0                                            
__________________________________________________________________________________________________
Min_TemperatureC (Inp

In [0]:
def train_fn(model_bytes):
    # Make sure pyarrow is referenced before anything else to avoid segfault due to conflict
    # with TensorFlow libraries.  Use `pa` package reference to ensure it's loaded before
    # functions like `deserialize_model` which are implemented at the top level.
    # See https://jira.apache.org/jira/browse/ARROW-3346
    pa

    import atexit
    import horovod.tensorflow.keras as hvd
    import os
    from petastorm import make_batch_reader
    from petastorm.tf_utils import make_petastorm_dataset
    import tempfile
    import tensorflow as tf
    import tensorflow.keras.backend as K
    import shutil

    # Horovod: initialize Horovod inside the trainer.
    hvd.init()

    # Horovod: pin GPU to be used to process local rank (one GPU per process), if GPUs are available.
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.visible_device_list = str(hvd.local_rank())
    K.set_session(tf.Session(config=config))

    # Horovod: restore from checkpoint, use hvd.load_model under the hood.
    model = deserialize_model(model_bytes, hvd.load_model)

    # Horovod: adjust learning rate based on number of processes.
    K.set_value(model.optimizer.lr, K.get_value(model.optimizer.lr) * hvd.size())

    # Horovod: print summary logs on the first worker.
    verbose = 2 if hvd.rank() == 0 else 0

    callbacks = [
        # Horovod: broadcast initial variable states from rank 0 to all other processes.
        # This is necessary to ensure consistent initialization of all workers when
        # training is started with random weights or restored from a checkpoint.
        hvd.callbacks.BroadcastGlobalVariablesCallback(root_rank=0),

        # Horovod: average metrics among workers at the end of every epoch.
        #
        # Note: This callback must be in the list before the ReduceLROnPlateau,
        # TensorBoard, or other metrics-based callbacks.
        hvd.callbacks.MetricAverageCallback(),

        # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
        # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
        # the first five epochs. See https://arxiv.org/abs/1706.02677 for details.
        hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=verbose),

        # Reduce LR if the metric is not improved for 10 epochs, and stop training
        # if it has not improved for 20 epochs.
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_exp_rmspe', patience=10, verbose=verbose),
        tf.keras.callbacks.EarlyStopping(monitor='val_exp_rmspe', mode='min', patience=20, verbose=verbose),
        tf.keras.callbacks.TerminateOnNaN()
    ]

    # Model checkpoint location.
    ckpt_dir = tempfile.mkdtemp()
    ckpt_file = os.path.join(ckpt_dir, 'checkpoint.h5')
    atexit.register(lambda: shutil.rmtree(ckpt_dir))

    # Horovod: save checkpoints only on the first worker to prevent other workers from corrupting them.
    if hvd.rank() == 0:
        callbacks.append(tf.keras.callbacks.ModelCheckpoint(ckpt_file, monitor='val_exp_rmspe', mode='min',
                                                            save_best_only=True))

    # Make Petastorm readers.
    with make_batch_reader('train_df.parquet' , num_epochs=None,
                           cur_shard=hvd.rank(), shard_count=hvd.size(),
                           hdfs_driver=PETASTORM_HDFS_DRIVER) as train_reader:
        with make_batch_reader('val_df.parquet' , num_epochs=None,
                               cur_shard=hvd.rank(), shard_count=hvd.size(),
                               hdfs_driver=PETASTORM_HDFS_DRIVER) as val_reader:
            # Convert readers to tf.data.Dataset.
            train_ds = make_petastorm_dataset(train_reader) \
                .apply(tf.data.experimental.unbatch()) \
                .shuffle(int(train_rows / hvd.size())) \
                .batch(BATCH_SIZE) \
                .map(lambda x: (tuple(getattr(x, col) for col in all_cols), tf.log(x.Sales)))

            val_ds = make_petastorm_dataset(val_reader) \
                .apply(tf.data.experimental.unbatch()) \
                .batch(BATCH_SIZE) \
                .map(lambda x: (tuple(getattr(x, col) for col in all_cols), tf.log(x.Sales)))

            history = model.fit(train_ds,
                                validation_data=val_ds,
                                steps_per_epoch=int(train_rows / BATCH_SIZE / hvd.size()),
                                validation_steps=int(val_rows / BATCH_SIZE / hvd.size()),
                                callbacks=callbacks,
                                verbose=verbose,
                                epochs=1)

    # Dataset API usage currently displays a wall of errors upon termination.
    # This global model registration ensures clean termination.
    # Tracked in https://github.com/tensorflow/tensorflow/issues/24570
    globals()['_DATASET_FINALIZATION_HACK'] = model

    if hvd.rank() == 0:
        with open(ckpt_file, 'rb') as f:
            return history.history, f.read()


In [0]:
# Create Spark session for training.
conf = SparkConf().setAppName('data_prep').set('spark.sql.shuffle.partitions', '64')

conf = SparkConf().setAppName('training').set('spark.sql.shuffle.partitions', '64')
if TRAINING_CLUSTER:
    conf.setMaster(TRAINING_CLUSTER)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Horovod: run training.
history, best_model_bytes = \
    horovod.spark.run(train_fn, args=(model_bytes,), num_proc=NUM_TRAINING_PROC, verbose=2)[0]

best_val_rmspe = min(history['val_exp_rmspe'])
print('Best RMSPE: %f' % best_val_rmspe)

# Write checkpoint.
with open(LOCAL_CHECKPOINT_FILE, 'wb') as f:
    f.write(best_model_bytes)
print('Written checkpoint to %s' % LOCAL_CHECKPOINT_FILE)

spark.stop()

Running 4 processes...


Exception in thread Thread-40:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/horovod/spark/__init__.py", line 81, in run_spark
    result = procs.mapPartitionsWithIndex(_make_mapper(driver.addresses(), settings)).collect()
  File "/content/spark-2.4.3-bin-hadoop2.7/python/pyspark/rdd.py", line 816, in collect
    sock_info = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/content/spark-2.4.3-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/content/spark-2.4.3-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/content/spark-2.4.3-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", l

Exception: ignored

In [0]:
model_bytes

In [0]:
!pip install h5py >=2.9.0

In [0]:
import h5py

In [0]:
h5py.__version__

In [0]:
!pip uninstall h5py

Uninstalling h5py-2.9.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/h5py-2.9.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-2.9.0


In [0]:
!pip install h5py>=2.9.0

In [0]:
import importlib
importlib.reload(h5py)
h5py.__version__

ImportError: ignored

In [0]:
model_bytes = serialize_model(model)


TypeError: ignored

In [0]:
h5py.__version__

'2.8.0'

In [0]:
!pip uninstall h5py

Uninstalling h5py-2.8.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/h5py-2.8.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-2.8.0


In [0]:
pip install h5py>=2.9.0

In [0]:
import importlib
importlib.reload(h5py)
h5py.__version__

ImportError: ignored